# DiCube Performance Comparison

Simple comparison of medical image formats:
- **DICOM vs DCB**: Traditional DICOM format vs DiCube compressed format
- **NIfTI vs DCB**: Neuroimaging format vs DiCube compressed format


In [1]:
# Performance measurement utilities
import contextlib
from pathlib import Path

@contextlib.contextmanager
def timer():
    """Context manager for timing operations"""
    import time
    start = time.time()
    yield lambda: time.time() - start
    
def get_folder_size(folder_path):
    """Calculate total size of files in a folder"""
    return sum(f.stat().st_size for f in Path(folder_path).glob("*"))

def get_file_size(file_path):
    """Get size of a single file"""
    return Path(file_path).stat().st_size

def format_size(size_bytes):
    """Format size in bytes to MB"""
    return size_bytes / 1024 / 1024

def print_results(title, results):
    """Print formatted performance results"""
    print(f"\n📊 {title}:")
    df = pd.DataFrame(results).round(3)
    print(df.to_string(index=False))

print("✅ Performance utilities defined")


✅ Performance utilities defined


In [2]:
import numpy as np
import pandas as pd
import time
import tempfile
import os
from pathlib import Path

# DICOM libraries
import pydicom
import dicube

# NIfTI libraries  
import nibabel as nib

os.chdir("../")

print("✅ All libraries imported successfully")



✅ All libraries imported successfully


In [3]:
def load_dicom_with_pydicom(dicom_folder):
    """Load DICOM files using pydicom and return pixel arrays"""
    dicom_files = sorted([f for f in Path(dicom_folder).glob("*.dcm")])
    datasets = []
    pixel_arrays = []
    
    for dcm_file in dicom_files:
        ds = pydicom.dcmread(dcm_file)
        datasets.append(ds)
        pixel_arrays.append(ds.pixel_array)
    
    return datasets, np.array(pixel_arrays)

def save_dicom_with_pydicom(datasets, pixel_arrays, output_folder):
    """Save DICOM files using pydicom"""
    os.makedirs(output_folder, exist_ok=True)
    
    for i, (ds, pixel_array) in enumerate(zip(datasets, pixel_arrays)):
        # Update pixel data
        ds.PixelData = pixel_array.tobytes()
        
        # Save to new location
        output_path = Path(output_folder) / f"slice_{i:04d}.dcm"
        ds.save_as(output_path)

print("✅ PyDICOM helper functions defined")


✅ PyDICOM helper functions defined


## DICOM Format Comparison


In [7]:
# Core functions for DICOM comparison
def test_pydicom_performance(dicom_folder, output_dir):
    """Test PyDICOM loading and saving performance"""
    with timer() as get_time:
        datasets, pixel_arrays = load_dicom_with_pydicom(dicom_folder)
    load_time = get_time()
    
    with timer() as get_time:
        save_dicom_with_pydicom(datasets, pixel_arrays, output_dir)
    save_time = get_time()
    
    folder_size = get_folder_size(output_dir)
    
    return {
        'data': (datasets, pixel_arrays),
        'load_time': load_time,
        'save_time': save_time,
        'file_size': folder_size,
        'shape': pixel_arrays.shape,
        'dtype': pixel_arrays.dtype
    }

def test_dicube_performance(dicom_folder, dcb_file):
    """Test DiCube loading and saving performance"""
    # Load from DICOM folder
    with timer() as get_time:
        dicube_image = dicube.load_from_dicom_folder(dicom_folder)
    load_time = get_time()
    
    # Save as DCB
    with timer() as get_time:
        dicube.save(dicube_image, str(dcb_file), file_type="s")
    save_time = get_time()
    
    # Load DCB file
    with timer() as get_time:
        loaded_image = dicube.load(str(dcb_file))
    dcb_load_time = get_time()
    
    file_size = get_file_size(dcb_file)
    data_consistent = np.array_equal(dicube_image.raw_image, loaded_image.raw_image)
    
    return {
        'original_image': dicube_image,
        'loaded_image': loaded_image,
        'load_time': dcb_load_time,
        'save_time': save_time,
        'file_size': file_size,
        'shape': dicube_image.shape,
        'dtype': dicube_image.raw_image.dtype,
        'data_consistent': data_consistent
    }

# Run DICOM comparison
dicom_folder = "testdata/dicom/sample_150"

if not os.path.exists(dicom_folder):
    print(f"❌ DICOM folder not found: {dicom_folder}")
else:
    print(f"✅ DICOM folder found: {dicom_folder}")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        
        # Test PyDICOM
        print("\n🔄 Testing PyDICOM...")
        pydicom_result = test_pydicom_performance(dicom_folder, pydicom_output)
        datasets, pixel_arrays = pydicom_result['data']
        
        # Test DiCube
        print("\n🔄 Testing DicomCubeImage...")
        dcb_file = tmpdir / "test.dcbs"
        dicube_result = test_dicube_performance(dicom_folder, dcb_file)
        
        
        # Prepare results for comparison
        results = [
            {
                'Method': 'PyDICOM',
                'Load Time (s)': pydicom_result['load_time'],
                'Save Time (s)': pydicom_result['save_time'],
                'File Size (MB)': format_size(pydicom_result['file_size']),
                'Compression Ratio': pixel_arrays.nbytes / pydicom_result['file_size']
            },
            {
                'Method': 'DCB (OJPH)',
                'Load Time (s)': dicube_result['load_time'],
                'Save Time (s)': dicube_result['save_time'],
                'File Size (MB)': format_size(dicube_result['file_size']),
                'Compression Ratio': pixel_arrays.nbytes / dicube_result['file_size']
            }
        ]
        
    print_results("DICOM Comparison Results", results)


✅ DICOM folder found: testdata/dicom/sample_150

🔄 Testing PyDICOM...

🔄 Testing DicomCubeImage...

📊 DICOM Comparison Results:
    Method  Load Time (s)  Save Time (s)  File Size (MB)  Compression Ratio
   PyDICOM          0.215          0.247          75.223              0.997
DCB (OJPH)          0.630          0.556          36.239              2.070


## NIfTI Format Comparison


In [ ]:
def load_nifti_with_nibabel(nifti_file):
    """Load NIfTI file using nibabel"""
    nii = nib.load(nifti_file)
    data = np.asarray(nii.dataobj, dtype=nii.dataobj.dtype)
    return nii, data

def save_nifti_with_nibabel(data, affine, output_file):
    """Save NIfTI file using nibabel"""
    nii = nib.Nifti1Image(data, affine)
    nib.save(nii, output_file)


# Core functions for NIfTI comparison
def test_nibabel_performance(nifti_file, output_file):
    """Test NiBabel loading and saving performance"""
    with timer() as get_time:
        nii, nifti_data = load_nifti_with_nibabel(nifti_file)
    load_time = get_time()
    
    with timer() as get_time:
        save_nifti_with_nibabel(nifti_data, nii.affine, output_file)
    save_time = get_time()
    
    file_size = get_file_size(output_file)
    
    return {
        'data': (nii, nifti_data),
        'load_time': load_time,
        'save_time': save_time,
        'file_size': file_size,
        'shape': nifti_data.shape,
        'dtype': nifti_data.dtype
    }

def test_dicube_nifti_performance(nifti_file, dcb_file):
    """Test DiCube NIfTI loading and saving performance"""
    # Load from NIfTI
    with timer() as get_time:
        dicube_image = dicube.load_from_nifti(nifti_file)
    load_time = get_time()
    
    # Save as DCB
    with timer() as get_time:
        dicube.save(dicube_image, str(dcb_file), file_type="s")
    save_time = get_time()
    
    # Load DCB file
    with timer() as get_time:
        loaded_image = dicube.load(str(dcb_file))
    dcb_load_time = get_time()
    
    file_size = get_file_size(dcb_file)
    data_consistent = np.array_equal(dicube_image.raw_image, loaded_image.raw_image)
    
    return {
        'original_image': dicube_image,
        'loaded_image': loaded_image,
        'load_time': dcb_load_time,
        'save_time': save_time,
        'file_size': file_size,
        'shape': dicube_image.shape,
        'dtype': dicube_image.raw_image.dtype,
        'data_consistent': data_consistent
    }

# Run NIfTI comparison
nifti_file = "testdata/nifti/CT_Philips.nii.gz"

if not os.path.exists(nifti_file):
    print(f"❌ NIfTI file not found: {nifti_file}")
else:
    print(f"✅ NIfTI file found: {nifti_file}")
    
    original_size = get_file_size(nifti_file)
    print(f"  Original file size: {format_size(original_size):.2f} MB")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        
        # Test NiBabel
        print("\n🔄 Testing NiBabel...")
        nibabel_output = tmpdir / "nibabel_output.nii.gz"
        nibabel_result = test_nibabel_performance(nifti_file, nibabel_output)
        
        nii, nifti_data = nibabel_result['data']
        
        # Test DiCube
        print("\n🔄 Testing DicomCubeImage...")
        dcb_file = tmpdir / "test_nifti.dcbs"
        dicube_result = test_dicube_nifti_performance(nifti_file, dcb_file)
        
        
        # Prepare results for comparison
        results = [
            {
                'Method': 'NiBabel',
                'Load Time (s)': nibabel_result['load_time'],
                'Save Time (s)': nibabel_result['save_time'],
                'File Size (MB)': format_size(nibabel_result['file_size']),
                'Compression Ratio': nifti_data.nbytes / nibabel_result['file_size']
            },
            {
                'Method': 'DCB (OJPH)',
                'Load Time (s)': dicube_result['load_time'],
                'Save Time (s)': dicube_result['save_time'],
                'File Size (MB)': format_size(dicube_result['file_size']),
                'Compression Ratio': nifti_data.nbytes / dicube_result['file_size']
            }
        ]
        
    print_results("NIfTI Comparison Results", results)


✅ NIfTI file found: testdata/nifti/CT_Philips.nii.gz
  Original file size: 7.45 MB

🔄 Testing NiBabel...

🔄 Testing DicomCubeImage...

📊 NIfTI Comparison Results:
    Method  Load Time (s)  Save Time (s)  File Size (MB)  Compression Ratio
   NiBabel          0.151          0.251           8.049              2.604
DCB (OJPH)          0.166          0.162           5.135              4.081


## Summary

This notebook demonstrates the performance comparison between traditional medical image formats and DiCube's compressed format:

**Key Findings:**
- **DCB files** typically offer significantly better compression ratios
- **DCB files** maintain data integrity (lossless compression)
- **DCB files** often have faster read/write times due to optimized compression
- **Single file storage** makes DCB files easier to manage and transfer

**Use Cases:**
- **Storage optimization**: Use DCB for long-term archival with space savings
- **Fast I/O**: DCB files are optimized for quick access
- **Compatibility**: DCB files can be converted back to DICOM/NIfTI when needed
